# Imports

In [1]:
# !pip install stop-words

In [2]:
import os
import sys
import pandas as pd
import numpy as np
import json
import datetime
import pickle

import nltk
from nltk import ne_chunk_sents, ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree

from collections import Counter
import operator

from stop_words import get_stop_words
from nltk.corpus import stopwords

# from nltk.stem.porter import PorterStemmer
# stemmer = PorterStemmer()

from nltk.stem.wordnet import WordNetLemmatizer
lemma = WordNetLemmatizer()

# Get Data

In [3]:
with open('pickled_files/raw_data.pickle', 'rb') as file:
    raw = pickle.load(file)

In [4]:
# Let's see what the dataframe looks like
raw.head()

,body_text,document_type,main_headline,print_headline,pub_date,score,section_name,snippet,source,type_of_material,word_count,writer_name
https://lens.blogs.nytimes.com/2017/06/21/handicapped-but-no-longer-invisible-andres-millan/,**********Andres Millan spent two years trave...,blogpost,"Dealing With Disabilities, but No Longer Invis...",None,2017-06-21,1.76345,None,For &#8220;Invisibles&#8221; Andres Millan spe...,The New York Times,Blog,897,David GONZALEZ
https://lens.blogs.nytimes.com/2017/08/22/combat-photographer-marine-ptsd-book/,**********My camera shutter whirred as a rock...,blogpost,"After Combat, a Photographer and a Marine Find...",None,2017-08-22,0.0711079,None,A photographer went from taking the picture of...,The New York Times,Blog,1080,Finbarr O'REILLY
https://www.nytimes.com/2017/06/01/nyregion/bronx-police-shooting-mental-illness.html,**********A week after a New York City police...,article,Officer’s Indictment Makes Good on Prosecutor’...,"For Bronx, Officer’s Indictment Makes Good on ...",2017-06-02,1.17119,None,Activists praised the murder indictment of Sgt...,The New York Times,News,1226,James McKINLEY
https://www.nytimes.com/2017/06/01/world/canada/nurse-killings-insulin.html,**********A former nurse pleaded guilty on Th...,article,Ex-Nurse Pleads Guilty to Killing 8 Patients a...,Former Nurse Pleads Guilty In 8 Killings In ...,2017-06-02,0.101218,Canada,Elizabeth Tracey Mae Wettlaufer admitted to in...,The New York Times,News,691,Matt STEVENS
https://www.nytimes.com/2017/06/02/learning/editorial-contest-winner-the-anguish-of-the-rich.html,**********We are honoring each of the Top 10 ...,article,Editorial Contest Winner | ‘The Anguish of the...,Editorial Contest Winner | ‘The Anguish of the...,2017-06-02,0.162545,None,We are honoring each of the Top 10 winners of ...,The New York Times,News,579,None


In [5]:
# Investigate an article
raw.iloc[5]['body_text']

' **********Welcome to the May 2017 edition of Teenagers in The Times, our monthly roundup of the news and feature stories about young people that have recently appeared across sections of NYTimes.com. Though we’ll be on summer hiatus for July and August, you can find a new edition in September. **********Our lesson plan and special activity sheet can be used with this or any other edition. **********_________ **********Education **********Math Champion Wins With Answer About Pecking Chicks **********A 13-year-old boy from Texas took less than a second to answer the word problem in the Raytheon Mathcounts National Competition. **********Florida University to Award Posthumous Degree to Trayvon Martin **********Florida Memorial University announced the degree in aviation would be given to his parents on his behalf on May 13 to recognize his love for flying. **********School Vouchers Aren’t Working, but Choice Is **********Charters and vouchers, often lumped together, are very different, 

## Check for Potential Index Duplicates

In [6]:
len(raw)

29547

In [7]:
# Count duplicates
len(raw[raw.duplicated()==True])

6

In [8]:
# Check duplicates
raw[raw.duplicated()==True]

,body_text,document_type,main_headline,print_headline,pub_date,score,section_name,snippet,source,type_of_material,word_count,writer_name
https://www.nytimes.com/2012/01/07/nyregion/fire-inspector-charged-in-day-care-scheme.html,**********A longtime Fire Department employee...,article,"Inspector at Fire Dept. Took Bribes, Officials...","Inspector At Fire Dept. Took Bribes, Officials...",2012-01-07,0.106512,None,A longtime department employee was charged wit...,The New York Times,News,611,Benjamin WEISER
https://query.nytimes.com/gst/fullpage.html?res=9D0DE4D71230F937A25752C0A9619C8B63,**********\n JAMES MURPHY'...,article,Helping Hand for Veterans Whose Lives Go Off T...,None,2007-01-14,1.18004,None,JAMES MURPHY's story may sound familiar to man...,The New York Times,News,695,Abraham STREEP
https://www.nytimes.com/2002/10/10/nyregion/democrats-fault-decision-to-move-mentally-ill.html,**********Democrats in the New York Assembly ...,article,Democrats Fault Decision to Move Mentally Ill,None,2002-10-10,1.47238,None,Democrats in the New York Assembly said yester...,The New York Times,News,338,Clifford LEVY
https://www.nytimes.com/1999/11/21/style/benefits-608920.html,**********Aiding Cancer Research **********MO...,article,BENEFITS,None,1999-11-21,0.0839854,None,Aiding Cancer Research MONDAY -- The American...,The New York Times,Schedule,1553,None
https://www.nytimes.com/1986/11/08/opinion/l-vatican-letter-fosters-antihomosexual-violence-435186.html,**********\n To the Editor...,article,Vatican Letter Fosters Antihomosexual Violence,None,1986-11-08,2.35904,None,To the Editor: The Vatican opinion that homo...,The New York Times,Letter,80,None
https://www.nytimes.com/1981/11/17/nyregion/news-summary-tuesday-november-17-1981.html,**********\n International...,article,"News Summary; TUESDAY, NOVEMBER 17, 1981",None,1981-11-17,0.101663,None,International A U.S. negotiating position on a...,The New York Times,News,805,None


In [9]:
# Drop duplicates
raw.drop_duplicates(inplace=True)

In [10]:
raw = raw[['body_text']].copy()

In [11]:
len(raw)

29541

# Initial Text Cleaning

In [12]:
raw['body_text'] = [''.join(x.strip().split('**********')) for x in raw['body_text']]
raw['body_text'] = [' '.join(x.split('_______')) for x in raw['body_text']]
raw['body_text'] = [''.join(x.split('\n                    ')) for x in raw['body_text']]
raw['body_text'] = [' '.join(x.split('         ')) for x in raw['body_text']]

# remove _________
# remove **********
# remove \n       
# remove tabs

In [13]:
# Let's check the same article as earlier
raw.iloc[5]['body_text']

'Welcome to the May 2017 edition of Teenagers in The Times, our monthly roundup of the news and feature stories about young people that have recently appeared across sections of NYTimes.com. Though we’ll be on summer hiatus for July and August, you can find a new edition in September. Our lesson plan and special activity sheet can be used with this or any other edition.  __ Education Math Champion Wins With Answer About Pecking Chicks A 13-year-old boy from Texas took less than a second to answer the word problem in the Raytheon Mathcounts National Competition. Florida University to Award Posthumous Degree to Trayvon Martin Florida Memorial University announced the degree in aviation would be given to his parents on his behalf on May 13 to recognize his love for flying. School Vouchers Aren’t Working, but Choice Is Charters and vouchers, often lumped together, are very different, claims this Opinion writer. The Broken Promises of Choice in New York City Schools The city’s high school a

# Further Cleaning with NLTK

## Excluded Named Entities

#### Get Named Entities

In [14]:
# Function to get the named entities from the text, then manual selection of entities to remove from bodytext

def get_continuous_chunks(named_entities,text):
    chunked = ne_chunk(pos_tag(word_tokenize(text)))
    prev = None
    continuous_chunk = []
    current_chunk = []
    for i in chunked:
        if type(i) == Tree:
            current_chunk.append(" ".join([token for token, pos in i.leaves()]))
        elif current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.append(named_entity)
                current_chunk = []
            else:
                continue
    named_entities += continuous_chunk

In [15]:
# # Retrieve pickled list of named entitites
# with open('pickled_files/list_of_named_entities.pickle', 'rb') as file:
#     named_entities = pickle.load(file)

In [16]:
# Get list of all named entities in articles
named_entities = []
article = 0
for a in raw['body_text']:
    get_continuous_chunks(named_entities,a)
    article = a
print(named_entities)

In [17]:
raw.tail()

,body_text
https://www.nytimes.com/1981/11/17/sports/sports-people-johnson-works-out.html,"Eddie Johnson, the Atlanta Hawks' star guard, ..."
https://www.nytimes.com/1981/11/18/garden/l-the-therapy-maze-097431.html,TO THE LIVING SECTION: Your generally fine dis...
https://www.nytimes.com/1981/11/18/nyregion/index-international.html,Brazil takes over town racked by family feud A...
https://www.nytimes.com/1981/11/18/nyregion/problems-grow-in-sheltering-homeless-in-city.html,New York City officials say that the rapidly g...
https://www.nytimes.com/1981/11/18/obituaries/herschel-alt-84-expert-on-young.html,"Herschel Alt, a specialist in mental health an..."


In [18]:
len(named_entities)

1069062

In [19]:
# Pickle list of named entities
with open('pickled_files/list_of_named_entities.pickle', 'wb') as file:
    pickle.dump(named_entities, file)

#### Get Dictionary with Counts

In [20]:
# Create dictionary from this list
named_entities_counts = Counter(named_entities)

In [21]:
len(named_entities_counts)

459596

In [22]:
named_entities_counts = sorted(named_entities_counts.items(), key=operator.itemgetter(1),reverse=True)

In [23]:
# Pickle dictionary of named entities counts
with open('pickled_files/dict_of_named_entities_counts.pickle', 'wb') as file:
    pickle.dump(named_entities_counts, file)

### Create Final List of Named Entities to Exclude

In [24]:
# # Get the pickle dictionary of named entities counts
# with open('pickled_files/dict_of_named_entities_counts.pickle', 'rb') as file:
#     named_entities_counts = pickle.load(file)

In [25]:
# Create final list of 1000 most occurring named entities to remove from text
common_entities = []
for i in np.arange(0,1000):
    common_entities.append(
#         [
            named_entities_counts[i][0]
#                                ,named_entities_counts[i][1]]
    )


In [26]:
# Investigate and use as input for entities to remove
common_entities

['New York',
 'University',
 'United States',
 'New York City',
 'American',
 'Manhattan',
 'Washington',
 'Health',
 'Brooklyn',
 'California',
 'Department',
 'America',
 'Mental Health',
 'Congress',
 'Republican',
 'Medicine',
 'Bronx',
 'New York Times',
 'Queens',
 'Senate',
 'New Jersey',
 'Los Angeles',
 'Children',
 'Chicago',
 'Calif.',
 'White House',
 'N.Y.',
 'Medicaid',
 'Center',
 'National Institute',
 'Boston',
 'WASHINGTON',
 'Texas',
 'House',
 'Florida',
 'Connecticut',
 'Supreme Court',
 'Federal',
 'Hospital',
 'AIDS',
 'New',
 'British',
 'English',
 'Democratic',
 'Americans',
 'Iraq',
 'N.J.',
 'Massachusetts',
 'Pennsylvania',
 'Clinton',
 'Mental Hygiene',
 'San Francisco',
 'Columbia University',
 'Long Island',
 'Harvard',
 'U.S.',
 'Obama',
 'State',
 'Army',
 'London',
 'French',
 'Columbia',
 'Bush',
 'Europe',
 'Ohio',
 'Times',
 'Philadelphia',
 'Medicare',
 'Mental',
 'New York State',
 'China',
 'Drug Administration',
 'Education',
 'Disease Control'

In [27]:
entities_to_remove = [
 'ALBANY',
 'AP',
 'Academy',
 'Adam',
 'Afghan',
 'Afghanistan',
 'Africa',
 'African',
 'Afterward',
 'Again',
 'Al Gore',
 'Al Qaeda',
 'Alabama',
 'Alaska',
 'Albany',
 'Alexandria',
 'Almost',
 'America',
 'American',
 'American American',
 'American Association',
 'American Journal',
 'Americans',
 'Americas',
 'Anderson',
 'Andrew',
 'Andrew Goldstein',
 'Andrew M. Cuomo',
 'Ann Arbor',
 'Apple',
 'April',
 'Arab',
 'Argentina',
 'Arizona',
 'Arkansas',
 'Arlington',
 'Asia',
 'Asian',
 'Atlanta',
 'Atlantic',
 'Atlantic City',
 'August',
 'Aurora',
 'Austin',
 'Australia',
 'Australian',
 'Austria',
 'Avenue',
 'BOSTON',
 'Baghdad',
 'Ballantine',
 'Baltimore',
 'Bantam',
 'Barack Obama',
 'Barbara',
 'Beijing',
 'Belgium',
 'Bellevue',
 'Belmont',
 'Berkeley',
 'Berlin',
 'Bethesda',
 'Bill Clinton',
 'Binghamton',
 'Birmingham',
 'Blasio',
 'Bob Dole',
 'Bosnia',
 'Boston',
 'Brazil',
 'Bridgeport',
 'Britain',
 'British',
 'Broadway',
 'Bronx',
 'Brooklyn',
 'Brown',
 'Buffalo',
 'Bush',
 'Bushwick',
 'CHICAGO',
 'Calif',
 'Calif.',
 'California',
 'California California',
 'Cambridge',
 'Camden',
 'Canada',
 'Canadian',
 'Capitol Hill',
 'Capt',
 'Caribbean',
 'Carter',
 'Central Park',
 'Chapel Hill',
 'Charles',
 'Charleston',
 'Charlotte',
 'Chelsea',
 'Chicago',
 'Child',
 'Chile',
 'China',
 'Chinatown',
 'Chinese',
 'Christ',
 'Christian',
 'Christopher',
 'Cincinnati',
 'City',
 'Clark',
 'Cleveland',
 'Clinton',
 'Cohen',
 'Colo.',
 'Colombia',
 'Colorado',
 'Columbia',
 'Columbine',
 'Columbus',
 'Conn',
 'Connecticut',
 'Cornell',
 'Creedmoor',
 'Cuba',
 'Cuban',
 'Cuomo',
 'D.C.',
 'DIOCESE OF BROOKLYN',
 'Dallas',
 'Daniel',
 'David',
 'David N. Dinkins',
 'Davis',
 'Delaware',
 'Denmark',
 'Denver',
 'Detroit',
 'Dinkins',
 'Disney',
 'Dominican Republic',
 'Don',
 'Donald',
 'Donald J. Trump',
 'Donald Trump',
 'Down',
 'Dr',
 'Dr.',
 'Duke',
 'Dutch',
 'East',
 'East Coast',
 'East Harlem',
 'East Side',
 'East Village',
 'Eastern',
 'Edward',
 'Egypt',
 'Eli Lilly',
 'Eliot Spitzer',
 'Elizabeth',
 'England',
 'English',
 'Europe',
 'European',
 'European Union',
 'Fifth Avenue',
 'First',
 'Fla',
 'Florida',
 'Fort Lauderdale',
 'Fox',
 'Fox News',
 'France',
 'Francisco',
 'Frank',
 'French',
 'Freud',
 'Freudian',
 'Frieden',
 'Further',
 'Gabrielle Giffords',
 'Gay',
 'George',
 'George Bush',
 'George E. Pataki',
 'George W. Bush',
 'Georgia',
 'German',
 'Germany',
 'Giants',
 'Giuliani',
 'Good',
 'Grand',
 'Great',
 'Great Neck',
 'Greece',
 'Greek',
 'Green',
 'Greenwich',
 'Greenwich Village',
 'HOW',
 'Haiti',
 'Hall',
 'Hamilton',
 'Harlem',
 'HarperCollins',
 'Harris',
 'Harry',
 'Hartford',
 'Harvard',
 'Harvard Medical School',
 'Hawaii',
 'Hill',
 'Hillary',
 'Hillary Clinton',
 'Hillary Rodham Clinton',
 'Hinckley',
 'Hitler',
 'Hollywood',
 'Hong Kong',
 'Houston',
 'How',
 'Howard',
 'Hudson River',
 'Huntington',
 'Hyperion',
 'INTERNATIONAL',
 'Idaho',
 'Illinois',
 'India',
 'Indian',
 'Indiana',
 'Indianapolis',
 'International',
 'Iowa',
 'Iran',
 'Iranian',
 'Iraq',
 'Iraq Afghanistan',
 'Iraq Iraq',
 'Iraqi',
 'Ireland',
 'Irish',
 'Irvine',
 'Islamic',
 'Israel',
 'Israeli',
 'Italian',
 'Italy',
 'Jack',
 'Jackson',
 'Jamaica',
 'James',
 'Japan',
 'Japanese',
 'Jersey',
 'Jersey City',
 'Jerusalem',
 'Jesus',
 'Jets',
 'Jewish',
 'Jim',
 'Jimmy Carter',
 'Joe',
 'John',
 'John F. Kennedy',
 'John McCain',
 'John W. Hinckley',
 'Johns Hopkins',
 'Johnson',
 'Jones',
 'Jordan',
 'Joseph',
 'Jr',
 'Jr.',
 'July',
 'Kansas',
 'Kansas City',
 'Kendra Webdale',
 'Kennedy',
 'Kentucky',
 'Knicks',
 'Knopf',
 'Koch',
 'Korea',
 'Korean',
 'LA',
 'LONDON',
 'LOS ANGELES',
 'Laden',
 'Lancet',
 'Las Vegas',
 'Latin',
 'Latin America',
 'Latino',
 'Lebanon',
 'Lee',
 'Lewis',
 'Lexington',
 'Lexington Avenue',
 'Likewise',
 'Lilly',
 'Lincoln',
 'Little',
 'Local',
 'London',
 'Long',
 'Long Island',
 'Long Island City',
 'Los Angeles',
 'Louisiana',
 'Lower East Side',
 'Lower Manhattan',
 'MICHAEL',
 'Madison',
 'Madison Avenue',
 'Main Street',
 'Maine',
 'Manhattan',
 'Marcos',
 'Mario M. Cuomo',
 'Mark',
 'Martin',
 'Martin Luther',
 'Mary',
 'Maryland',
 'Massachusetts',
 'Mayor Koch',
 'Mayor Michael',
 'McDonald',
 'Md',
 'Me',
 'Memphis',
 'Mets',
 'Mexican',
 'Mexico',
 'Miami',
 'Michael',
 'Michael R. Bloomberg',
 'Michigan',
 'Middle East',
 'Middletown',
 'Midtown',
 'Midwest',
 'Mike',
 'Miller',
 'Milwaukee',
 'Minneapolis',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Money',
 'Montana',
 'Montreal',
 'Moore',
 'Morris',
 'Moscow',
 'Mother',
 'Mr',
 'Mr.',
 'Mr.',
 'Mr. Bloomberg',
 'Mr. Brown',
 'Mr. Bush',
 'Mr. Clinton',
 'Mr. Cuomo',
 'Mr. Cuomo Mr. Cuomo',
 'Mr. Giuliani',
 'Mr. Goldstein',
 'Mr. Hinckley',
 'Mr. Hinckley Mr. Hinckley',
 'Mr. Johnson',
 'Mr. Kennedy',
 'Mr. Koch',
 'Mr. Miller',
 'Mr. Obama',
 'Mr. Pataki',
 'Mr. Reagan',
 'Mr. Smith',
 'Mr. Trump',
 'Mr. de Blasio',
 'Mrs',
 'Mrs.',
 'Ms',
 'Ms.',
 'Muslim',
 'N.C.',
 'N.J.',
 'N.Y.',
 'NATO',
 'NEW',
 'NY',
 'NYC',
 'NYT',
 'Nancy',
 'Nashville',
 'Nassau',
 'Nassau County',
 'Navy',
 'Nebraska',
 'Nelson',
 'Netherlands',
 'Nevada',
 'New',
 'New England',
 'New England Journal',
 'New Hampshire',
 'New Haven',
 'New Jersey',
 'New Mexico',
 'New Orleans',
 'New Rochelle',
 'New York',
 'New York City',
 'New York City New York',
 'New York New Jersey',
 'New York New York',
 'New York New York City',
 'New York New York Times Neediest Cases Fund',
 'New York Post',
 'New York State',
 'New York Times',
 'New York Times Neediest Cases Fund',
 'New Yorker',
 'New Yorkers',
 'New Zealand',
 'Newark',
 'Newton',
 'Newtown',
 'Nigeria',
 'No',
 'Nobody',
 'North',
 'North America',
 'North Carolina',
 'North Korea',
 'Northeast',
 'Northern',
 'Norton',
 'Norwalk',
 'Norway',
 'Note',
 'Oakland',
 'Obama',
 'Obamacare',
 'Often',
 'Ohio',
 'Oklahoma',
 'Oklahoma City',
 'Ontario',
 'Orange County',
 'Oregon',
 'Orlando',
 'Pa',
 'Pacific',
 'Pakistan',
 'Palestinian',
 'Palestinians',
 'Palo Alto',
 'Paris',
 'Park',
 'Park Avenue',
 'Park Avenue South',
 'Park Slope',
 'Parker',
 'Parkland',
 'Parks',
 'Pataki',
 'Paterson',
 'Paul',
 'Paxil',
 'Pennsylvania',
 'Persian Gulf',
 'Peru',
 'Peter',
 'Philadelphia',
 'Philippines',
 'Phoenix',
 'Physicians',
 'Pittsburgh',
 'Poland',
 'Polish',
 'Poor',
 'Port',
 'Portland',
 'Post',
 'Princeton',
 'Providence',
 'Puerto Rico',
 'Putnam',
 'Queens',
 'Rather',
 'Reagan',
 'Reagan Administration',
 'Reuters',
 'Rhode Island',
 'Richard',
 'Richmond',
 'Rikers',
 'Rikers Island',
 'Riverdale',
 'Robert',
 'Rochester',
 'Rockland',
 'Rockville',
 'Roe',
 'Rome',
 'Ronald Reagan',
 'Rudolph W. Giuliani',
 'Russia',
 'Russian',
 'Rutgers',
 'Rye',
 'S.C.',
 'SAN FRANCISCO',
 'Sacramento',
 'Sam',
 'San',
 'San Antonio',
 'San Diego',
 'San Francisco',
 'Santa Barbara',
 'Santa Monica',
 'Sarah',
 'Saudi Arabia',
 'Scarsdale',
 'Schuster',
 'Science Times',
 'Scotland',
 'Scott',
 'Seattle',
 'Second',
 'Sgt',
 'Sigmund Freud',
 'Silicon Valley',
 'Simon',
 'Simpson',
 'Smith',
 'SoHo',
 'Someone',
 'Soon',
 'South',
 'South Africa',
 'South America',
 'South Bronx',
 'South Carolina',
 'South Dakota',
 'South Korea',
 'Southern',
 'Southern California',
 'Soviet',
 'Soviet Union',
 'Spain',
 'Spanish',
 'Springfield',
 'Sr',
 'Sr.',
 'St.',
 'St. John',
 'St. Louis',
 'St. Martin',
 'St. Paul',
 'Stamford',
 'Stanford',
 'Staten Island',
 'Stony Brook',
 'Street',
 'Suffolk',
 'Suffolk County',
 'Sullivan',
 'Sundays',
 'Super Bowl',
 'Susan',
 'Sweden',
 'Swedish',
 'Swiss',
 'Switzerland',
 'Syracuse',
 'Syria',
 'THE',
 'Taiwan',
 'Taliban',
 'Tampa',
 'Tennessee',
 'Tex',
 'Tex.',
 'Texas',
 'Thailand',
 'Thomas',
 'Thomas R. Frieden',
 'Time',
 'Times',
 'Times Square',
 'Tokyo',
 'Tom',
 'Too',
 'Toronto',
 'Town',
 'Trenton',
 'Trump',
 'Tucson',
 'Turkey',
 'U.S.',
 'Ukraine',
 'United',
 'United Nations',
 'United States',
 'United States American',
 'United States United States',
 'Upper East Side',
 'Upper West Side',
 'Utah',
 'Va',
 'Valhalla',
 'Vermont',
 'Vienna',
 'Vietnam',
 'Virginia',
 'WASHINGTON',
 'Wade',
 'Wall Street Journal',
 'Wards Island',
 'Warner',
 'Wash.',
 'Washington',
 'Washington Post',
 'West',
 'West Bank',
 'West Coast',
 'West Side',
 'West Virginia',
 'Westchester',
 'Westchester County',
 'Western',
 'Westport',
 'Which',
 'White',
 'White Plains',
 'William',
 'Williams',
 'Williamsburg',
 'Wilson',
 'Wisconsin',
 'Yale',
 'Yankees',
 'Yonkers',
 'York',
 'Young',
 'Zoloft',
 'â\x80\x9d'
]

In [28]:
len(entities_to_remove)

651

In [29]:
entities_to_remove = sorted(entities_to_remove)

In [30]:
entities_to_remove

['ALBANY',
 'AP',
 'Academy',
 'Adam',
 'Afghan',
 'Afghanistan',
 'Africa',
 'African',
 'Afterward',
 'Again',
 'Al Gore',
 'Al Qaeda',
 'Alabama',
 'Alaska',
 'Albany',
 'Alexandria',
 'Almost',
 'America',
 'American',
 'American American',
 'American Association',
 'American Journal',
 'Americans',
 'Americas',
 'Anderson',
 'Andrew',
 'Andrew Goldstein',
 'Andrew M. Cuomo',
 'Ann Arbor',
 'Apple',
 'April',
 'Arab',
 'Argentina',
 'Arizona',
 'Arkansas',
 'Arlington',
 'Asia',
 'Asian',
 'Atlanta',
 'Atlantic',
 'Atlantic City',
 'August',
 'Aurora',
 'Austin',
 'Australia',
 'Australian',
 'Austria',
 'Avenue',
 'BOSTON',
 'Baghdad',
 'Ballantine',
 'Baltimore',
 'Bantam',
 'Barack Obama',
 'Barbara',
 'Beijing',
 'Belgium',
 'Bellevue',
 'Belmont',
 'Berkeley',
 'Berlin',
 'Bethesda',
 'Bill Clinton',
 'Binghamton',
 'Birmingham',
 'Blasio',
 'Bob Dole',
 'Bosnia',
 'Boston',
 'Brazil',
 'Bridgeport',
 'Britain',
 'British',
 'Broadway',
 'Bronx',
 'Brooklyn',
 'Brown',
 'Buffa

In [31]:
with open('pickled_files/entities_to_remove.pickle', 'wb') as file:
    pickle.dump(entities_to_remove, file)

In [32]:
# with open('pickled_files/entities_to_remove.pickle', 'rb') as file:
#     pickle.load(file)

### Remove Named Entities from Articles

In [33]:
# Function for removal
def remove_entities(article):
    for entity in entities_to_remove:
        if ' '+entity+' ' in article:
            article = article.replace(entity+' ','') 
        elif ' '+entity+'.' in article:
            article = article.replace(' '+entity,'')
        elif ' '+entity+',' in article:
            article = article.replace(' '+entity,'')
        elif ' '+entity+':' in article:
            article = article.replace(' '+entity,'')
        elif ' '+entity+'-' in article:
            article = article.replace(' '+entity,'')
        elif ' '+entity+';' in article:
            article = article.replace(' '+entity,'')
        elif ' '+entity+'"' in article:
            article = article.replace(' '+entity,'')
        elif ' '+entity+"'" in article:
            article = article.replace(' '+entity,'')
        elif ' '+entity+"]" in article:
            article = article.replace(' '+entity,'')
        elif ' '+entity+")" in article: # added later
            article = article.replace(' '+entity,'')
        elif ' '+entity+"?" in article:
            article = article.replace(' '+entity,'')
        elif ' '+entity+"!" in article: # added later
            article = article.replace(' '+entity,'')
        elif '"'+entity+' ' in article:
            article = article.replace(entity+' ','')
        elif "'"+entity+' ' in article:
            article = article.replace(entity+' ','')
        elif "["+entity+' ' in article:
            article = article.replace(entity+' ','')
        elif "("+entity+' ' in article: # added later
            article = article.replace(entity+' ','')
        elif "["+entity+']' in article:
            article = article.replace(entity,'')
        elif "("+entity+')' in article: # added later
            article = article.replace(entity,'')
        elif "'"+entity+"'" in article:
            article = article.replace(entity,'')
        elif '"'+entity+'"' in article:
            article = article.replace(entity,'')
    return(article)

In [34]:
# Remove final named entities from articles
raw['body_text_noent'] = [remove_entities(x) for x in raw['body_text']]

In [35]:
with open('pickled_files/raw_data_cleaned_named_ent_removed.pickle', 'wb') as file:
    pickle.dump(raw, file)

## Tokenize

In [37]:
# Split articles into words
raw['tokenized_text'] = [word_tokenize(x) for x in raw['body_text_noent']]

## Remove Punctuation

In [38]:
# Remove punctuation
raw['tokenized_nopunc'] = [[word for word in x if word.isalpha()] for x in raw['tokenized_text']]

# Alternative method, if we'd be interested in keeping numbers as well:
# import string
# exclude = set(string.punctuation) 
# punc_free = ''.join(ch for ch in stop_free if ch not in exclude)

In [39]:
# Check
raw.iloc[3455]['tokenized_nopunc']

['Good',
 'Wednesday',
 'morning',
 'from',
 'Washington',
 'where',
 'the',
 'National',
 'Transportation',
 'Safety',
 'Board',
 'starts',
 'its',
 'investigation',
 'into',
 'the',
 'deadly',
 'train',
 'accident',
 'and',
 'confirmation',
 'hearings',
 'get',
 'underway',
 'for',
 'Ashton',
 'as',
 'defense',
 'secretary',
 'Senator',
 'Mitch',
 'McConnell',
 'of',
 'celebrates',
 'his',
 'home',
 'state',
 'over',
 'a',
 'barrel',
 'of',
 'bourbon',
 'Representative',
 'Aaron',
 'office',
 'has',
 'the',
 'town',
 'talking',
 'and',
 'Senator',
 'Thom',
 'Tillis',
 'in',
 'the',
 'name',
 'of',
 'liberty',
 'defends',
 'the',
 'freedom',
 'of',
 'restaurant',
 'employees',
 'who',
 'want',
 'to',
 'wash',
 'their',
 'hands',
 'We',
 'start',
 'with',
 'Jeb',
 'Bush',
 'who',
 'is',
 'in',
 'and',
 'looking',
 'more',
 'and',
 'more',
 'like',
 'a',
 'presidential',
 'candidate',
 'Greetings',
 'from',
 'Motown',
 'Former',
 'Gov',
 'Jeb',
 'of',
 'delivers',
 'his',
 'first',
 'pr

## Remove Capitalization

In [40]:
# Remove capitalization
raw['tokenized_nopunc_lower'] = [[word.lower() for word in x] for x in raw['tokenized_nopunc']]

In [41]:
# Check
raw.iloc[3455]['tokenized_nopunc_lower']

['good',
 'wednesday',
 'morning',
 'from',
 'washington',
 'where',
 'the',
 'national',
 'transportation',
 'safety',
 'board',
 'starts',
 'its',
 'investigation',
 'into',
 'the',
 'deadly',
 'train',
 'accident',
 'and',
 'confirmation',
 'hearings',
 'get',
 'underway',
 'for',
 'ashton',
 'as',
 'defense',
 'secretary',
 'senator',
 'mitch',
 'mcconnell',
 'of',
 'celebrates',
 'his',
 'home',
 'state',
 'over',
 'a',
 'barrel',
 'of',
 'bourbon',
 'representative',
 'aaron',
 'office',
 'has',
 'the',
 'town',
 'talking',
 'and',
 'senator',
 'thom',
 'tillis',
 'in',
 'the',
 'name',
 'of',
 'liberty',
 'defends',
 'the',
 'freedom',
 'of',
 'restaurant',
 'employees',
 'who',
 'want',
 'to',
 'wash',
 'their',
 'hands',
 'we',
 'start',
 'with',
 'jeb',
 'bush',
 'who',
 'is',
 'in',
 'and',
 'looking',
 'more',
 'and',
 'more',
 'like',
 'a',
 'presidential',
 'candidate',
 'greetings',
 'from',
 'motown',
 'former',
 'gov',
 'jeb',
 'of',
 'delivers',
 'his',
 'first',
 'pr

## Remove Stop Words - Generic List python stop_words // nltk.stopwords

In [42]:
# Let's start from two generic lists:
# - nltk stopwords
# - stopwords

In [43]:
en_stop = sorted(list(set(get_stop_words('en') + stopwords.words('english'))))

# py_en_stop = get_stop_words('en')
# nltk_en_stop = list(set(stopwords.words('english')))

In [44]:
len(en_stop)

211

In [45]:
# Remove stop words
raw['tokenized_nopunc_lower_nostop'] = [[word for word in x if not word in en_stop] for x in raw['tokenized_nopunc_lower']]

In [46]:
# Check
raw.iloc[29000]['tokenized_nopunc_lower_nostop']

['washington',
 'select',
 'commission',
 'immigration',
 'refugee',
 'policy',
 'decided',
 'recommend',
 'change',
 'law',
 'bars',
 'certain',
 'classes',
 'aliens',
 'including',
 'communists',
 'homosexuals',
 'states',
 'commission',
 'decided',
 'effect',
 'sidestep',
 'one',
 'main',
 'functions',
 'expected',
 'perform',
 'apparently',
 'subject',
 'controversial',
 'final',
 'report',
 'issued',
 'commission',
 'urges',
 'congress',
 'criteria',
 'excluding',
 'aliens',
 'avoids',
 'making',
 'specific',
 'proposals',
 'subject',
 'congress',
 'creating',
 'commission',
 'three',
 'years',
 'ago',
 'asked',
 'comprehensive',
 'review',
 'immigration',
 'law',
 'together',
 'recommendations',
 'simplify',
 'clarify',
 'commission',
 'voted',
 'recommend',
 'general',
 'amnesty',
 'illegal',
 'aliens',
 'states',
 'comprehensive',
 'revision',
 'visas',
 'allocated',
 'stiff',
 'penalties',
 'employers',
 'hire',
 'illegal',
 'aliens',
 'future',
 'final',
 'meeting',
 'commiss

In [47]:
# Check
len(raw.iloc[29000]['tokenized_nopunc_lower_nostop'])

467

In [48]:
# Now, gathering more stopwords from looking at some samples, let's remove some more

In [49]:
extra_stop_words = [
    'big','small','low','high',
    'none',
    'may',
    'among',
    'within',
    'don','t',
    'day',
    'etc',
    'around',
    'frequent',
    'including',
    'even',
    'can',
    'likely',
    'will',
    'like',
    'today',
    'bit',
    'put',
    'aim',
    's',
    'got',
    'really',
    'huge',
    'see',
    'almost',
    'already',
    'much',
    'recent',   #
    'many',
    'change',    #
    'changes',       #
    'someone',
    'said',
    'says',
    'gives',
    'give',
#     'people',
    'new',
    'say',
    'least','first','last','second',
    'one','two',
    'go',
    'goes',
    'take',
    'going',
    'taking',
    'just',
    'can'
    'cannot',
    'keep',
    'keeps',
    'also',
    'done',
    'good',
    'get',
    'without',
    'told',
    'might',
    'time',
    'unable',  #
    'able',  #
    'know',
    'end',
    'now',
    'want',
    'didn',
    'back',
    'doesn',
    'couldn',
    'since',
    'shouldn',
    'seen',
    'works',
    'zero',
    'every',
    'each',
    'other',
    'ever',
    'neither',
    'll',
    'mr',
    'ms',
    'mrs',
    'think',
    'tomorrow',
    'way',
    'still',
    'know',
    'later',
    'fine',    #
    'let',
    'went',
    'night',
    've',
    'must',
    'act',  #
    're',
    'c','b', 'a',
    'done',
    'began',
    'ones',
    'm',
    'soon',
    'word',
    'along',
    'main',
    'q',
    'lot',
    'e', 'd',
    'entire',
    'year',
    'mean',
    'means',
    'important',
    'always',
    'something',
    'rather',
    'either',
    'makes',
    'make',
    'uses',
    'use',
    'enough',
    'w','d',
    'never',
    'giving',
    'o',
    'involve',
    'involes',
    'involving',
    'little',
    'inside',
    'sat',
    'third','fourth','fifth','sixth',
    'next',
    'given',
    'million','billion','millions','billions',
    'option',
    'options',
    'full',
    'complete',
    'need',
    'needs',
    'set',
    'manage',
    'sets',
    'manages',
    'bring','brings','brought',
    'try','tries','tried'
    'week',
    'former',
    'monday','tuesday','wednesday','thursday','friday','saturday','sunday',
    'spent','spend', 'spends',
    'month','months',
    'send','sends','sent',
    'went',
    'january','february','march','april','may','june','july','august','september','october','november','december',
    'allow',
    'process',
#     'old',
    'times',
    'nearly',
    'looking','looks','look',
    'thinly',
    'becoming',
    'stay','stays',
    'took','takes','take',
    'types', 'type',
    'thought', 'though',
    'idea',
    'clear','clearly',
    'behind',
    'half',
    'us',
    'less',
    'claim','claims',
    'long', 'short',
    'smaller','larger','bigger','largest','biggest','smallest','longer','shorter','short','long',
    'extreme','severe',
    'largely',
    'anymore',
    'years',
    'spoke',
    'give','gave','given','gives',
    'reportedly','supposedly','alledgedly',
    'please',
    'received','receive','receives',
    'longtime',
    'best',
    'existing',
    'putting','put','puts',
    
    'whose',
    'yesterday',
    
    
    
    'thing',   #added later
    'week',
    'another',
    'month',
    'day',
    'come',
    

]

In [50]:
# Remove extra stop words
raw['tokenized_nopunc_lower_nostop_extra'] = [[word for word in x if not word in extra_stop_words] for x in raw['tokenized_nopunc_lower_nostop']]

In [51]:
# Check
len(raw.iloc[800]['tokenized_nopunc_lower_nostop_extra'])

428

In [52]:
len(raw.iloc[800]['tokenized_text'])

998

In [53]:
# From spot checking it seems we've cut each article down by ~60% (up to 65%), which is great

## Lemmatitization

In [54]:
# This might be a better alternative to stemming, to combine singular and plurals etc.

## From some spot testing on the cleaned text so far, 
## the variety in words goes down by ~2-15 words per article from lemmatizing (2-8%)

In [55]:
raw['tokenized_nopunc_lower_nostop_extra_lemmatized'] = [[lemma.lemmatize(word) for word in x] for x in raw['tokenized_nopunc_lower_nostop_extra']]

In [56]:
# Check
len(set((raw.iloc[1030]['tokenized_nopunc_lower_nostop_extra_lemmatized'])))

1591

In [57]:
# Check
len(set((raw.iloc[1030]['tokenized_nopunc_lower_nostop_extra'])))

1732

In [58]:
# Check
len(set(raw.iloc[1030]['tokenized_text']))

2317

## Stemming - Decided Against

## Check

In [59]:
for i in raw['tokenized_nopunc_lower_nostop_extra_lemmatized'].sample(5):
    print(i,'\n')

['pompano', 'beach', 'president', 'came', 'hospitalized', 'victim', 'horrific', 'school', 'shooting', 'offered', 'thanks', 'doctor', 'nurse', 'helped', 'wounded', 'declaring', 'job', 'incredible', 'asked', 'talked', 'victim', 'added', 'indeed', 'sad', 'happen', 'paying', 'respect', 'medical', 'professional', 'broward', 'health', 'hospital', 'wife', 'melania', 'visited', 'law', 'enforcement', 'official', 'officer', 'hoped', 'getting', 'credit', 'deserved', 'response', 'shooting', 'left', 'dead', 'injured', 'hospital', 'parent', 'thankful', 'job', 'broward', 'county', 'sheriff', 'office', 'joined', 'gov', 'rick', 'marco', 'rubio', 'official', 'added', 'young', 'victim', 'great', 'shape', 'considering', 'relished', 'bantering', 'praising', 'law', 'enforcement', 'official', 'marveled', 'speed', 'responder', 'rushed', 'wounded', 'hospital', 'extended', 'hearty', 'thanks', 'responder', 'president', 'natural', 'consolation', 'publicly', 'address', 'grief', 'sorrow', 'gripping', 'shocked', 'co

In [60]:
raw.head()

,body_text,body_text_noent,tokenized_text,tokenized_nopunc,tokenized_nopunc_lower,tokenized_nopunc_lower_nostop,tokenized_nopunc_lower_nostop_extra,tokenized_nopunc_lower_nostop_extra_lemmatized
https://lens.blogs.nytimes.com/2017/06/21/handicapped-but-no-longer-invisible-andres-millan/,Andres Millan spent two years traveling around...,Andres Millan spent two years traveling around...,"[Andres, Millan, spent, two, years, traveling,...","[Andres, Millan, spent, two, years, traveling,...","[andres, millan, spent, two, years, traveling,...","[andres, millan, spent, two, years, traveling,...","[andres, millan, traveling, environs, photogra...","[andres, millan, traveling, environs, photogra..."
https://lens.blogs.nytimes.com/2017/08/22/combat-photographer-marine-ptsd-book/,My camera shutter whirred as a rocket-propelle...,My camera shutter whirred as a rocket-propelle...,"[My, camera, shutter, whirred, as, a, rocket-p...","[My, camera, shutter, whirred, as, a, grenade,...","[my, camera, shutter, whirred, as, a, grenade,...","[camera, shutter, whirred, grenade, wobbled, t...","[camera, shutter, whirred, grenade, wobbled, t...","[camera, shutter, whirred, grenade, wobbled, t..."
https://www.nytimes.com/2017/06/01/nyregion/bronx-police-shooting-mental-illness.html,A week after a New York City police sergeant s...,A week after a police sergeant shot and killed...,"[A, week, after, a, police, sergeant, shot, an...","[A, week, after, a, police, sergeant, shot, an...","[a, week, after, a, police, sergeant, shot, an...","[week, police, sergeant, shot, killed, woman, ...","[week, police, sergeant, shot, killed, woman, ...","[week, police, sergeant, shot, killed, woman, ..."
https://www.nytimes.com/2017/06/01/world/canada/nurse-killings-insulin.html,A former nurse pleaded guilty on Thursday to k...,A former nurse pleaded guilty on Thursday to k...,"[A, former, nurse, pleaded, guilty, on, Thursd...","[A, former, nurse, pleaded, guilty, on, Thursd...","[a, former, nurse, pleaded, guilty, on, thursd...","[former, nurse, pleaded, guilty, thursday, kil...","[nurse, pleaded, guilty, killing, eight, patie...","[nurse, pleaded, guilty, killing, eight, patie..."
https://www.nytimes.com/2017/06/02/learning/editorial-contest-winner-the-anguish-of-the-rich.html,We are honoring each of the Top 10 winners of ...,We are honoring each of the Top 10 winners of ...,"[We, are, honoring, each, of, the, Top, 10, wi...","[We, are, honoring, each, of, the, Top, winner...","[we, are, honoring, each, of, the, top, winner...","[honoring, top, winners, fourth, annual, stude...","[honoring, top, winners, annual, student, edit...","[honoring, top, winner, annual, student, edito..."


# Troubleshooting

In [62]:
# One article has a body in which all words are clumped together, let's find and drop it
raw[raw['body_text'].str.contains('improvementsthat')]['body_text']
# looks like the transcript has some errors

https://www.nytimes.com/2008/02/21/us/politics/21text-demdebate.html    The following is a transcript of the Democrati...
Name: body_text, dtype: object

In [63]:
# Let's drop that one
raw.drop('https://www.nytimes.com/2008/02/21/us/politics/21text-demdebate.html',axis=0,inplace=True)

In [64]:
# Another article has a body in which all words are clumped together, let's find and drop it
raw[raw['body_text'].str.contains('agoaloriented')]['body_text']
# looks like the transcript has some errors

https://www.nytimes.com/1981/10/04/magazine/the-carey-s-of-albany.html    Richard J. Meislin is chief of The Times's Alb...
Name: body_text, dtype: object

In [65]:
# Let's drop that one
raw.drop('https://www.nytimes.com/1981/10/04/magazine/the-carey-s-of-albany.html',axis=0,inplace=True)

# Corpora Check

# Export to Pickle and Move On To Modeling

In [66]:
with open('pickled_files/clean_data_full.pickle', 'wb') as file:
    pickle.dump(raw, file)

In [67]:
# with open('pickled_files/clean_data_full.pickle', 'rb') as file:
#     raw = pickle.load(file)

In [68]:
# Drop interim colums
raw_small = pd.DataFrame(raw['tokenized_nopunc_lower_nostop_extra_lemmatized'])
raw_small.rename(columns={'tokenized_nopunc_lower_nostop_extra_lemmatized':'article_text'},inplace=True)

In [69]:
with open('pickled_files/clean_data_small.pickle', 'wb') as file:
    pickle.dump(raw_small, file)